# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\TI\AppData\Local\Temp\graphlab_server_1482319805.log.0


This non-commercial license of GraphLab Create for academic use is assigned to tinsae.gizachew@astu.edu.et and will expire on July 29, 2017.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 4         | 5        | 0.000002  | 1.984000     | 6847166.848958     | 384842.383767 |

| 5         | 6        | 0.000002  | 2.039000     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 2.087000     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

Find what features had non-zero weight.

In [6]:
model_all.get("coefficients").print_rows(18)

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

h<sub>0</sub>[<b>x</b> <sub>i</sub>]    <b> intercept </b>

h<sub>3</sub>[<b>x</b> <sub>i</sub>]    <b> bathroom</b>

h<sub>4</sub>[<b>x</b> <sub>i</sub>]    <b> sqft.Living</b>

h<sub>5</sub>[<b>x</b> <sub>i</sub>]    <b> sqft.Living_sqrt</b>

h<sub>13</sub>[<b>x</b> <sub>i</sub>]    <b> grade</b>

h<sub>14</sub>[<b>x</b> <sub>i</sub>]    <b> sqft_above</b>

#### TGA Attempt

In [7]:
#model_all is a linear reg object
#model_all.get("coef") is an sframe
#An sframe contains a set of dictionaries. Keys correspond to the columns of the sframe

def see_non_zero(model, wanna_print):
    count=0
    for i in model.get("coefficients"):
        if(i["value"] != 0.0):
            if(wanna_print):
                print "%-20s%-20s"%(i["name"], i["value"])
            count=count+1
    return count

see_non_zero(model_all, True)

(intercept)         274873.05595        
bathrooms           8468.53108691       
sqft_living         24.4207209824       
sqft_living_sqrt    350.060553386       
grade               842.068034898       
sqft_above          20.0247224171       


6

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions=model.predict(data)    
    # Then compute the residuals/errors
    RSS=outcome-predictions
    # Then square and add them up
    RSS=(RSS*RSS).sum()
    return(RSS)    
min=1e100
op=1e100
for L in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=L, verbose=False)
    RSS=get_residual_sum_of_squares(model,validation, validation['price'])
    print RSS
    if(RSS<min):
        min=RSS
        op=L
        
print "op is: %s and RSS is %d"% (op,RSS)

6.25766285142e+14
6.25766285362e+14
6.25766286058e+14
6.25766288257e+14
6.25766295212e+14
6.25766317206e+14
6.25766386761e+14
6.25766606749e+14
6.25767302792e+14
6.25769507644e+14
6.25776517727e+14
6.25799062845e+14
6.25883719085e+14
op is: 10.0 and RSS is 625883719085425


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

6.2588 e+14

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [10]:
print see_non_zero(model, False)
#all are non-zero(18 of them)

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [11]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [12]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [13]:
non_zeros=[]
high=18
low=0
op_high=0
op_low=0
for L in np.logspace(8,10, num=20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=L, verbose=False)
    #see_non_zero(model, False)
    v=model['coefficients']['value'].nnz()

    if(v<high and v>=7):
        high=v
        op_high=L
    if(v>low and v<=7):
        low=v
        op_low=L
    non_zeros.append(v)
    
print non_zeros
print low, high
print op_low, op_high

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]
6 10
3792690190.73 2976351441.63


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [14]:
l1_penalty_min = op_low
l1_penalty_max = op_high

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

#### TGA Answer

In [15]:
print l1_penalty_min
print l1_penalty_max

3792690190.73
2976351441.63


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [16]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [17]:
min=1e100
op=1e100
i=0
round=-1
best_model=None
for L in np.linspace(l1_penalty_min,l1_penalty_max,20):
    
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=L, verbose=False)
    RSS=get_residual_sum_of_squares(model,validation, validation['price'])
    v=model['coefficients']['value'].nnz()
    print "Round%2d Lamda is %-3.f, RSS=%-10s"%(i+1,L,RSS)
    #Now we are searching not only the lowest error,but also sparsity of 7
    if(RSS<min and (v is 7)):
        min=RSS
        op=L
        round=i
        best_model=model
        print "new minimum with sparsity 7 found"
        
    i=i+1
print "op(lamda) is: %s and RSS is %sfound @ round %s"% (op,min,round+1)

Round 1 Lamda is 3792690191, RSS=1.08186759232e+15
Round 2 Lamda is 3749724993, RSS=1.07763277558e+15
Round 3 Lamda is 3706759796, RSS=1.07350454959e+15
Round 4 Lamda is 3663794599, RSS=1.06946433543e+15
Round 5 Lamda is 3620829401, RSS=1.0657076895e+15
Round 6 Lamda is 3577864204, RSS=1.06079953176e+15
new minimum with sparsity 7 found
Round 7 Lamda is 3534899007, RSS=1.05599273534e+15
new minimum with sparsity 7 found
Round 8 Lamda is 3491933809, RSS=1.05114762561e+15
new minimum with sparsity 7 found
Round 9 Lamda is 3448968612, RSS=1.04693748875e+15
new minimum with sparsity 7 found
Round10 Lamda is 3406003415, RSS=1.04323723787e+15
Round11 Lamda is 3363038218, RSS=1.03855473594e+15
Round12 Lamda is 3320073020, RSS=1.03461690923e+15
Round13 Lamda is 3277107823, RSS=1.02824799221e+15
Round14 Lamda is 3234142626, RSS=1.01829878055e+15
Round15 Lamda is 3191177428, RSS=1.00847716702e+15
Round16 Lamda is 3148212231, RSS=9.98783211266e+14
Round17 Lamda is 3105247034, RSS=9.89328342459e+1

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

#### TGA Answer

#As done in the earlier cells, it should be between

3792690190.73 &

2976351441.63

<b>Q1</b>

==>3448968612.16<==

<b>Q2</b>

In [24]:
##
print "The following are the 7 most relvant features"
for i in best_model.get("coefficients"):
    if(i['value']!=0):
        print i['name']
best_model['coefficients']['value'].nnz()

The following are the 7 most relvant features
(intercept)
bedrooms
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


7

#### TGA, Just for Fun
##### custom linspace, (for log space it is just taking the log10 of each output of linspace)

In [19]:
#do it other time